In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from scipy.stats import norm
import time

In [2]:
API_KEY = {'key':'DIVV75B4'}
KEY = 'DIVV75B4'
BASE = 'http://localhost:9999/v1/'

In [3]:
tickers_1 = ['RTM1C45', 'RTM1P45', 'RTM1C46', 'RTM1P46', 'RTM1C47', 
             'RTM1P47', 'RTM1C48', 'RTM1P48', 'RTM1C49', 'RTM1P49',
             'RTM1C50', 'RTM1P50', 'RTM1C51', 'RTM1P51', 'RTM1C52',
             'RTM1P52', 'RTM1C53', 'RTM1P53', 'RTM1C54', 'RTM1P54'
]
tickers_2 = ['RTM2C45', 'RTM2P45', 'RTM2C46', 'RTM2P46', 'RTM2C47',
             'RTM2P47', 'RTM2C48', 'RTM2P48', 'RTM2C49', 'RTM2P49', 
             'RTM2C50', 'RTM2P50', 'RTM2C51', 'RTM2P51', 'RTM2C52',
             'RTM2P52', 'RTM2C53', 'RTM2P53', 'RTM2C54', 'RTM2P54']

all_tickers = ['RTM1C45', 'RTM1P45', 'RTM1C46', 'RTM1P46', 'RTM1C47', 
             'RTM1P47', 'RTM1C48', 'RTM1P48', 'RTM1C49', 'RTM1P49',
             'RTM1C50', 'RTM1P50', 'RTM1C51', 'RTM1P51', 'RTM1C52',
             'RTM1P52', 'RTM1C53', 'RTM1P53', 'RTM1C54', 'RTM1P54',
             'RTM2C45', 'RTM2P45', 'RTM2C46', 'RTM2P46', 'RTM2C47',
             'RTM2P47', 'RTM2C48', 'RTM2P48', 'RTM2C49', 'RTM2P49', 
             'RTM2C50', 'RTM2P50', 'RTM2C51', 'RTM2P51', 'RTM2C52',
             'RTM2P52', 'RTM2C53', 'RTM2P53', 'RTM2C54', 'RTM2P54'
               
]
strike_prices = [45, 46, 47, 48, 49, 50, 51, 52, 53, 54]

In [18]:
class Trader_2:
    
    def __init__(self, BASE, API_KEY, KEY, strike_prices):
        
        self.BASE = BASE
        self.API_KEY = API_KEY
        self.KEY = KEY
        self.strike_prices = strike_prices
        self.delta_limit = 10000
        
        self.started_vol = False
        self.used_call_vol = None
        self.used_put_vol = None
        self.strategy_vol = None
        self.used_extra_opt = None
  

        
        self.started_end_straddle = False
        self.opt_str_1 = None
        self.opt_str_2 = None
        
        self.started_arb_p2 = False
        self.strategy_arb_p2 = None
        self.opt_for_arb_1 = None
        self.opt_for_arb_2 = None
        self.quant_p2 = 0
        
        self.period = 1
        self.tick = 0
        self.vol = 0.2
        self.r = 0
        
             
        self.iv_list = []
        self.tick_list = []
        
    def update_iv(self):
        
        data, iv = trader.implied_vol_enhanced()
        self.iv_list.append(iv)
        self.tick_list.append(self.tick)
        
    def open_vol(self):
        
        low_forecast, high_forecast = self.get_forecast_volatility()
        mean_forecast = (low_forecast + high_forecast)/2
        
        spot, spot_pos, _ = self.get_data()
        data, vol_atm = self.implied_vol_enhanced()
        atm = min(self.strike_prices, key=lambda x:abs(x-spot))
        cur_vol = vol_atm
        
            
        if self.period == 2 or self.tick > 235:
            data = data[data['stop_period'] == 2]

        if cur_vol > mean_forecast:

            self.used_extra_opt = data.sort_values(by=['vega', 'delta'], key=abs).iloc[0]['ticker']

            self.started_vol = True
            self.strategy_vol = 'short'
            self.used_call_vol = f'RTM2C{atm}'
            self.used_put_vol = f'RTM2P{atm}'
            
            data, vol_atm = self.implied_vol_enhanced()
            data = data[data['stop_period'] >= self.period]
            data['arb'] = data['IV'] - vol_atm
            
            found = False
            
            if (data['arb'] > 0).sum() >= 2: # if there are 2 diff type and same direction
                
                if not data[(data['arb'] > 0)&(data['call'])].empty and\
                   not data[(data['arb'] > 0)&(~data['call'])].empty:
                    
                    self.used_call_vol = data[(data['arb'] > 0)&(data['call'])].loc[data[(data['arb'] > 0)&(data['call'])]['arb'].idxmax()]['ticker']
                    self.used_put_vol = data[(data['arb'] > 0)&(~data['call'])].loc[data[(data['arb'] > 0)&(~data['call'])]['arb'].idxmax()]['ticker']
                    found = True
                    
            if not found and (data['arb'] > 0).any():
                
                for_arb = data.loc[data['arb'].idxmax()]
                
                if for_arb['ticker'][4] == 'C':
                    
                    self.used_call_vol = for_arb['ticker']
                    self.used_put_vol = f'RTM2P{self.used_call_vol[-2:]}'
                    
                else:
                    
                    self.used_put_vol = for_arb['ticker']
                    self.used_call_vol = f'RTM2C{self.used_put_vol[-2:]}'
                    
                
    
            for i in range(5):

                self.post_order(self.used_call_vol, 'SELL', 100)
                self.post_order(self.used_put_vol, 'SELL', 100)
            
            self.get_tick()
            self.delta_hedge(full=True)
            data, vol_atm = self.implied_vol_enhanced()
            data['arb'] = data['IV'] - vol_atm
            if (data['arb'] < 0).any():
                self.used_extra_opt = data.loc[data['arb'].idxmin()]['ticker']
            if self.used_extra_opt == self.used_call_vol or self.used_extra_opt == self.used_put_vol:
                self.used_extra_opt = data.sort_values(by=['vega', 'delta'], key=abs).iloc[0]['ticker']
                

            for i in range(3):

                self.post_order(self.used_extra_opt, 'BUY', 100)
                self.post_order(self.used_extra_opt, 'BUY', 100)
                self.post_order(self.used_call_vol, 'SELL', 100)
                self.post_order(self.used_put_vol, 'SELL', 100)
                
            self.post_order(self.used_extra_opt, 'BUY', 100)
            self.post_order(self.used_call_vol, 'SELL', 50)
            self.post_order(self.used_put_vol, 'SELL', 50)


            print(f'Opened position vol; tick: {self.tick}')
            print(f'Sold {self.used_call_vol} and {self.used_put_vol}; IV: {round(cur_vol,2)}. Extra opt:{self.used_extra_opt}')
            print(f'Low: {low_forecast}, High: {high_forecast}, Past: {self.vol}')
            print(f'Spot price: {spot}')
            print('-----------')    

        elif cur_vol < mean_forecast:

            self.used_extra_opt = data.sort_values(by=['vega', 'delta'], key=abs).iloc[0]['ticker']

            self.started_vol = True
            self.strategy_vol = 'long'
            self.used_call_vol = f'RTM2C{atm}'
            self.used_put_vol = f'RTM2P{atm}'
            
            data, vol_atm = self.implied_vol_enhanced()
            data = data[data['stop_period'] >= self.period]
            data['arb'] = data['IV'] - vol_atm
            
            found = False
            
            if (data['arb'] < 0).sum() >= 2: # if there are 2 diff type and same direction
                
                if not data[(data['arb'] < 0)&(data['call'])].empty and\
                   not data[(data['arb'] < 0)&(~data['call'])].empty:
                    
                    self.used_call_vol = data[(data['arb'] < 0)&(data['call'])].loc[data[(data['arb'] < 0)&(data['call'])]['arb'].idxmin()]['ticker']
                    self.used_put_vol = data[(data['arb'] < 0)&(~data['call'])].loc[data[(data['arb'] < 0)&(~data['call'])]['arb'].idxmin()]['ticker']
                    found = True
                    
            if not found and (data['arb'] < 0).any():
                
                for_arb = data.loc[data['arb'].idxmin()]
                
                if for_arb['ticker'][4] == 'C':
                    self.used_call_vol = for_arb['ticker']
                else:
                    self.used_put_vol = for_arb['ticker']
  
            
            for i in range(5):

                self.post_order(self.used_call_vol, 'BUY', 100)
                self.post_order(self.used_put_vol, 'BUY', 100)
            
            self.get_tick()
            self.delta_hedge(full=True)
            
            data, vol_atm = self.implied_vol_enhanced()
            data['arb'] = data['IV'] - vol_atm
            
            if (data['arb'] > 0).any():
                self.used_extra_opt = data.loc[data['arb'].idxmax()]['ticker']
            if self.used_extra_opt == self.used_call_vol or self.used_extra_opt == self.used_put_vol:
                self.used_extra_opt = data.sort_values(by=['vega', 'delta'], key=abs).iloc[0]['ticker']
            

            for i in range(3):

                self.post_order(self.used_extra_opt, 'SELL', 100)
                self.post_order(self.used_extra_opt, 'SELL', 100)
                self.post_order(self.used_call_vol, 'BUY', 100)
                self.post_order(self.used_put_vol, 'BUY', 100)
                
            self.post_order(self.used_extra_opt, 'SELL', 100)
            self.post_order(self.used_call_vol, 'BUY', 50)
            self.post_order(self.used_put_vol, 'BUY', 50)

            print(f'Opened position vol; tick: {self.tick}')
            print(f'Bought {self.used_call_vol} and {self.used_put_vol}; IV: {round(cur_vol,2)}. Extra opt:{self.used_extra_opt}')
            print(f'Low: {low_forecast}, High: {high_forecast}, Past: {self.vol}')
            print(f'Spot price: {spot}')
            print('-----------')  
            
        
    def close_vol(self):
        
        data, vol_atm = self.implied_vol_enhanced()
        target_vol = self.vol
        cur_vol = vol_atm
        
        print(f'Cur vol : {round(cur_vol,2)}, Target: {self.vol}')
        
        if self.strategy_vol == 'long' and\
        ((cur_vol >= target_vol) or self.tick % 75 >= 30 or (self.period==2 and self.tick % 75 >= 25))and\
            data[data['ticker'] == self.used_call_vol]['IV'].values[0] ==\
            data[data['ticker'] == self.used_put_vol]['IV'].values[0] ==\
            data[data['ticker'] == self.used_extra_opt]['IV'].values[0]:
            
            
            for i in range(5):

                self.post_order(self.used_call_vol, 'SELL', 100)
                self.post_order(self.used_put_vol, 'SELL', 100)
                
            self.get_tick()
            self.delta_hedge(full=True)
            
            #time.sleep(0.5)

            for i in range(3):

                self.post_order(self.used_extra_opt, 'BUY', 100)
                self.post_order(self.used_extra_opt, 'BUY', 100)
                self.post_order(self.used_call_vol, 'SELL', 100)
                self.post_order(self.used_put_vol, 'SELL', 100)
                
            self.post_order(self.used_extra_opt, 'BUY', 100)
            self.post_order(self.used_call_vol, 'SELL', 50)
            self.post_order(self.used_put_vol, 'SELL', 50)


            self.started_vol = False

            print(f'Closed position vol')
            
        elif self.strategy_vol == 'short' and \
        ((cur_vol <= target_vol) or self.tick % 75 >= 50) and\
            data[data['ticker'] == self.used_call_vol]['IV'].values[0] ==\
            data[data['ticker'] == self.used_put_vol]['IV'].values[0]==\
            data[data['ticker'] == self.used_extra_opt]['IV'].values[0]:
            
         
            for i in range(5):

                self.post_order(self.used_call_vol, 'BUY', 100)
                self.post_order(self.used_put_vol, 'BUY', 100)
                
            
            self.get_tick()
            self.delta_hedge(full=True)
            
            #time.sleep(0.5)
            
            for i in range(3):

                self.post_order(self.used_extra_opt, 'SELL', 100)
                self.post_order(self.used_extra_opt, 'SELL', 100)
                self.post_order(self.used_call_vol, 'BUY', 100)
                self.post_order(self.used_put_vol, 'BUY', 100)
                
            self.post_order(self.used_extra_opt, 'SELL', 100)
            self.post_order(self.used_call_vol, 'BUY', 50)
            self.post_order(self.used_put_vol, 'BUY', 50)

            self.started_vol = False

            print(f'Closed position vol')

            
    def open_arb_p2(self):
        """
        If 2 options with same type and diff direction => buy one short other
        If exists arb for second month => use calendar spread
        If exists arb for first month => use closest option opposite from spot
        Prefer second month over first month
        """

        data, vol_atm = self.implied_vol_enhanced()
        data['arb'] = data['IV'] - vol_atm
        self.quant_p2 = 0
        
        if (data['arb'].abs() >= 0.02).any():
            
            time.sleep(3.5)
            self.get_tick()
            
            data, vol_atm = self.implied_vol_enhanced()
            data = data[data['stop_period'] >= self.period]
            data['arb'] = data['IV'] - vol_atm
            
            found = False
            
            if (data['arb'] > 0).any() and (data['arb'] < 0).any(): # if there are 2 same type and diff direction
                
                if not data[(data['arb'] > 0)&(data['call'])].empty and\
                   not data[(data['arb'] < 0)&(data['call'])].empty:
                    
                    for_arb_more = data[(data['arb'] > 0)&(data['call'])].iloc[0]
                    for_arb_less = data[(data['arb'] < 0)&(data['call'])].iloc[0]
                    self.opt_for_arb_1 = for_arb_more['ticker']
                    self.opt_for_arb_2 = for_arb_less['ticker']
                    found = True
                    self.strategy_arb_p2 = 'short'
                    
                    print('Arb p2 options found')
                    
                elif not data[(data['arb'] > 0)&(~data['call'])].empty and\
                   not data[(data['arb'] < 0)&(~data['call'])].empty:
                    
                    for_arb_more = data[(data['arb'] > 0)&(~data['call'])].iloc[0]
                    for_arb_less = data[(data['arb'] < 0)&(~data['call'])].iloc[0]
                    self.opt_for_arb_1 = for_arb_more['ticker']
                    self.opt_for_arb_2 = for_arb_less['ticker']
                    found = True
                    self.strategy_arb_p2 = 'short'
                    print('Arb p2 options found')

                    
            if not found:
                
                if not data[(data['arb'] > 0)&(data['stop_period'] == 2)].empty and self.period == 1:
                    
                    for_arb = data[data['stop_period'] == 2].loc[data[data['stop_period'] == 2]['arb'].abs().idxmax()]
                    self.opt_for_arb_1 = for_arb['ticker']
                    self.opt_for_arb_2 = f'RTM1{self.opt_for_arb_1[4]}{self.opt_for_arb_1[-2:]}'
                    
                    if for_arb['arb'] > 0:
                        self.strategy_arb_p2 = 'short'
                    elif for_arb['arb'] < 0:
                        self.strategy_arb_p2 = 'long'
                    print('Arb p2 options found. Calendar spread')

                elif self.period == 2:
                    
                    for_arb = data.loc[data['arb'].abs().idxmax()]
                    spot, _, _ = self.get_data()
                    self.opt_for_arb_1 = for_arb['ticker']
                    k_arb = int(for_arb['ticker'][-2:])
                    call_ = self.opt_for_arb_1[4]
                    
                    if k_arb < 54 and k_arb > 45:
                        if spot > k_arb:
                            self.opt_for_arb_2 = f'RTM2{call_}{k_arb-1}'
                        else:
                            self.opt_for_arb_2 = f'RTM2{call_}{k_arb+1}'
                    elif k_arb == 54:
                        self.opt_for_arb_2 = f'RTM2{call_}{k_arb-1}'
                    elif k_arb == 45:
                        self.opt_for_arb_2 = f'RTM2{call_}{k_arb+1}'
                        
                    if for_arb['arb'] > 0:
                        self.strategy_arb_p2 = 'short'
                    elif for_arb['arb'] < 0:
                        self.strategy_arb_p2 = 'long'
                    print('Arb p2 options found. Period 2')

                        
                elif self.period == 1:
                    
                    for_arb = data.loc[data['arb'].abs().idxmax()]
                    spot, _, _ = self.get_data()
                    self.opt_for_arb_1 = for_arb['ticker']
                    k_arb = int(for_arb['ticker'][-2:])
                    call_ = self.opt_for_arb_1[4]
                    
                    if k_arb < 54 and k_arb > 45:
                        if spot > k_arb:
                            self.opt_for_arb_2 = f'RTM1{call_}{k_arb-1}'
                        else:
                            self.opt_for_arb_2 = f'RTM1{call_}{k_arb+1}'
                    elif k_arb == 54:
                        self.opt_for_arb_2 = f'RTM1{call_}{k_arb-1}'
                    elif k_arb == 45:
                        self.opt_for_arb_2 = f'RTM1{call_}{k_arb+1}'
                        
                    if for_arb['arb'] > 0:
                        self.strategy_arb_p2 = 'short'
                    elif for_arb['arb'] < 0:
                        self.strategy_arb_p2 = 'long'
                    print('Arb p2 options found. Period 1')

                    
            if self.strategy_arb_p2 == 'short':
                
                self.started_arb_p2 = True
                
                for i in range(10):
                    
                    self.post_order(self.opt_for_arb_1, 'SELL', 100)
                    self.post_order(self.opt_for_arb_2, 'BUY', 100)
                    self.quant_p2 += 1
                    
                    self.get_tick()
                    data, vol_atm = self.implied_vol_enhanced()
                    if data[data['ticker'] == self.opt_for_arb_1]['IV'].values[0] == \
                    data[data['ticker'] == self.opt_for_arb_2]['IV'].values[0]:
                        break
                        
                    if i == 4:
                        self.delta_hedge(full=True)
                
                print(f'Opened arb p2. Sold {self.opt_for_arb_1} Bought {self.opt_for_arb_2}')

            elif self.strategy_arb_p2 == 'long':

                self.started_arb_p2 = True

                for i in range(10):
                    
                    self.post_order(self.opt_for_arb_1, 'BUY', 100)
                    self.post_order(self.opt_for_arb_2, 'SELL', 100)
                    self.quant_p2 += 1
                    
                    self.get_tick()
                    data, vol_atm = self.implied_vol_enhanced()
                    if data[data['ticker'] == self.opt_for_arb_1]['IV'].values[0] == \
                    data[data['ticker'] == self.opt_for_arb_2]['IV'].values[0]:
                        break
                        
                    if i == 4:
                        self.delta_hedge(full=True)

                print(f'Opened arb p2. Bought {self.opt_for_arb_1} Sold {self.opt_for_arb_2}')
            
    def close_arb_p2(self):
        
        data, vol_atm = self.implied_vol_enhanced()
        _, spot_pos, _ = self.get_data()
        
        if data[data['ticker'] == self.opt_for_arb_1]['IV'].values[0] == \
            data[data['ticker'] == self.opt_for_arb_2]['IV'].values[0] or self.tick % 75 >= 57:
            
            if self.strategy_arb_p2 == 'long':

                for i in range(self.quant_p2):
                
                    self.post_order(self.opt_for_arb_1, 'SELL', 100)
                    self.post_order(self.opt_for_arb_2, 'BUY', 100)
                    
                    self.get_tick()
                    data, vol_atm = self.implied_vol_enhanced()
                    while data[data['ticker'] == self.opt_for_arb_1]['IV'].values[0] != \
                        data[data['ticker'] == self.opt_for_arb_2]['IV'].values[0]:
                        time.sleep(0.2)
                        self.get_tick()
                        data, vol_atm = self.implied_vol_enhanced()
                    
                    if i == 4:
                        self.get_tick()
                        self.delta_hedge(full=True)
                
                print('Closed arb p2.')
                self.started_arb_p2 = False
                self.strategy_arb_p2 = None
                    
            elif self.strategy_arb_p2 == 'short':

                for i in range(self.quant_p2):
                
                    self.post_order(self.opt_for_arb_1, 'BUY', 100)
                    self.post_order(self.opt_for_arb_2, 'SELL', 100)
                    
                    if i == 4:
                        self.get_tick()
                        self.delta_hedge(full=True)
                        
                    self.get_tick()
                    data, vol_atm = self.implied_vol_enhanced()
                    while data[data['ticker'] == self.opt_for_arb_1]['IV'].values[0] != \
                        data[data['ticker'] == self.opt_for_arb_2]['IV'].values[0]:
                        time.sleep(0.2)
                        self.get_tick()
                        data, vol_atm = self.implied_vol_enhanced()
                    
                    
                print('Closed arb p2.')
                self.started_arb_p2 = False
                self.strategy_arb_p2 = None
            

            
    def end_straddle(self):
        
        spot, spot_pos, data = self.get_data()
        
        atm = min(self.strike_prices, key=lambda x:abs(x-spot))
        
        if atm < 53 and atm > 46:
            self.opt_str_1 = f'RTM2P{int(atm-1)}'
            self.opt_str_2 = f'RTM2C{int(atm+1)}'
        elif atm == 53 or atm == 46:
            self.opt_str_1 = f'RTM2P{int(atm-1)}'
            self.opt_str_2 = f'RTM2C{int(atm+1)}'
            
        if self.opt_str_1:
            
            for i in range(5):
                self.post_order(self.opt_str_1, 'SELL', 100)
                self.post_order(self.opt_str_2, 'SELL', 100)

            print(f'Opened end straddle with {self.opt_str_1} and {self.opt_str_2}')
            print(f'Spot: {spot}')
            
    
    def delta_hedge(self, full=False):
        
        data, vol_atm = self.implied_vol_enhanced()
        iv = vol_atm
        data = data[data['stop_period'] >= self.period]
        spot, spot_pos, _ = self.get_data()
        
        cur_delta = 100 * (data[data['ticker'] != 'RTM']['delta'] *\
                    data[data['ticker'] != 'RTM']['position']).sum() + \
                    spot_pos
        if full:
            delta_limit = 2000
            delta_target = 0
        else:
            delta_limit = self.delta_limit
            delta_target = 0.75 * delta_limit
            
        if cur_delta > delta_limit:
            
            delta_rtm = int(cur_delta - delta_target)
            while delta_rtm > 10000:
                self.post_order('RTM', 'SELL', 10000)
                delta_rtm -= 10000
            self.post_order('RTM', 'SELL', int(delta_rtm))
            print(f'Cur delta: {round(cur_delta)}. Hedging')
                
        elif cur_delta < -delta_limit:
            
            delta_rtm = int(abs(cur_delta + delta_target))
            while delta_rtm > 10000:
                self.post_order('RTM', 'BUY', 10000)
                delta_rtm -= 10000
            self.post_order('RTM', 'BUY', int(delta_rtm))
            print(f'Cur delta: {round(cur_delta)}. Hedging')
            
        return cur_delta
        
            

    ######################## Parsing functions ########################
    
    def get_delta_limit(self):
        
        resp = pd.DataFrame(requests.get(f'{self.BASE}news', params=self.API_KEY).json())
        resp = resp[resp['headline'].str.contains('Delta Limit')]
        if not resp.empty:
            self.delta_limit = int(''.join(re.findall('[0-9]+,[0-9]+', resp['body'].values[0])[0].split(',')))
        else:
            self.delta_limit = -1
            
    def get_real_volatility(self):
        
        resp = requests.get(f'{self.BASE}news', params=self.API_KEY) 
        news = pd.DataFrame(resp.json())
        news = news[news['headline'].str.contains('Announcement')]
        if not news.empty:
            real_volatility = int(re.findall('[0-9]+', news['body'].values[0])[0])
            self.vol = real_volatility/100

    
    def get_forecast_volatility(self):
        
        resp = requests.get(f'{self.BASE}news', params=self.API_KEY) 
        news = pd.DataFrame(resp.json())
        news = news[news['headline'].str.contains('News')]
        if not news.empty:
            low_forecast = int(re.findall('[0-9]+% ~', news['body'].values[0])[0].split('%')[0])
            high_forecast = int(re.findall('~ [0-9]+%', news['body'].values[0])[0].split('~ ')[1][:-1])
            return low_forecast/100, high_forecast/100
        else:
            return -1, -1 
    
    #################### Option pricing functions ######################
    
    def black_scholes(self, s, k, t, r, vol, call):
        
        """
        Takes time in days to expiration, not tick
        """
        
        d1 = (np.log(s / k) + (r + vol ** 2 / 2) * t) / (vol * np.sqrt(t))
        d2 = d1 - vol * np.sqrt(t)
        if call:
            price = s * norm.cdf(d1) - k * norm.cdf(d2) * np.exp(-r * t)
        else:
            price = k * norm.cdf(-d2) * np.exp(-r * t) - s * norm.cdf(-d1)
            
        if price < 0.02:
                price = 0.02

        return price
    
    def delta(self, s, k, t, r, vol, call):
        
        d1 = (np.log(s/k) + (r + vol ** 2 / 2) *t) / (vol * np.sqrt(t))
        if call:
            delta = np.exp(-r * t) * norm.cdf(d1)
        else:
            delta = np.exp(-r * t) * (norm.cdf(d1) - 1)
        return delta
    
    def implied_volatility(self, s, k, t, r, market_price, init_vol, call):
        
        test_price = self.black_scholes(s,k,t,r,init_vol,call)
        d1 = (np.log(s/k) + (r + init_vol ** 2 / 2) * t) / (init_vol * np.sqrt(t))
        vega = s * np.sqrt(t) * norm.pdf(d1)
        vol = init_vol

        for i in range(200):
            vol = vol + (market_price - test_price) / vega
            if vol < 0.00001:
                vol = 0.00001
                break
            d1 =(np.log(s/k) + (r + vol ** 2 / 2) * t) / (vol * np.sqrt(t))
            vega = s * np.sqrt(t) * norm.cdf(d1)
            test_price = self.black_scholes(s,k,t,r,vol,call)
            diff = abs(market_price - test_price)
            if diff < 0.00001:
                break
        return vol
    
    def implied_vol_enhanced(self):
        
        t = (600 - ((self.period - 1) * 300 + self.tick))/3600
        r = self.r
        
        
        s, spot_pos, data = trader.get_data()
        data = data[data['stop_period'] >= self.period]
        k_atm = min(strike_prices, key=lambda x:abs(x-s))
        
        price_atm_call = data[data['ticker']==f'RTM2C{k_atm}']['last'].values[0]
        price_atm_put =  data[data['ticker']==f'RTM2P{k_atm}']['last'].values[0]
        
        vol_atm_call = self.implied_volatility(s, k_atm, t, r, price_atm_call, self.vol, 1)
        vol_atm_put = self.implied_volatility(s, k_atm, t, r, price_atm_put, self.vol, 0)
        
        if abs(vol_atm_call - vol_atm_put) <= 0.02:
            vol_atm = vol_atm_call
            
        else:
            
            check_vol_call = 0
            check_vol_put = 1
            
            if k_atm > 45 and k_atm < 54:
                
                check_vol_call += abs(self.black_scholes(s, (k_atm - 1), t, r, vol_atm_call, 1) - data[data['ticker']==f'RTM2C{k_atm-1}']['last'].values[0]) <= 0.03
                check_vol_call += abs(self.black_scholes(s, (k_atm - 1), t, r, vol_atm_call, 0) - data[data['ticker']==f'RTM2P{k_atm-1}']['last'].values[0]) <= 0.03
                check_vol_call += 1
                check_vol_call += abs(self.black_scholes(s, k_atm, t, r, vol_atm_call, 0) - data[data['ticker']==f'RTM2P{k_atm}']['last'].values[0]) <= 0.03
                check_vol_call += abs(self.black_scholes(s, (k_atm + 1), t, r, vol_atm_call, 1) - data[data['ticker']==f'RTM2C{k_atm+1}']['last'].values[0]) <= 0.03
                check_vol_call += abs(self.black_scholes(s, (k_atm + 1), t, r, vol_atm_call, 0) - data[data['ticker']==f'RTM2P{k_atm+1}']['last'].values[0]) <= 0.03
                
                check_vol_put += abs(self.black_scholes(s, (k_atm - 1), t, r, vol_atm_put, 1) - data[data['ticker']==f'RTM2C{k_atm-1}']['last'].values[0]) <= 0.03
                check_vol_put += abs(self.black_scholes(s, (k_atm - 1), t, r, vol_atm_put, 0) - data[data['ticker']==f'RTM2P{k_atm-1}']['last'].values[0]) <= 0.03
                check_vol_put += abs(self.black_scholes(s, k_atm, t, r, vol_atm_put, 1) - data[data['ticker']==f'RTM2C{k_atm}']['last'].values[0]) <= 0.03
                check_vol_put += 1
                check_vol_put += abs(self.black_scholes(s, (k_atm + 1), t, r, vol_atm_put, 1) - data[data['ticker']==f'RTM2C{k_atm+1}']['last'].values[0]) <= 0.03
                check_vol_put += abs(self.black_scholes(s, (k_atm + 1), t, r, vol_atm_put, 0) - data[data['ticker']==f'RTM2P{k_atm+1}']['last'].values[0]) <= 0.03
                
            elif k_atm == 45:
                
                check_vol_call += 1
                check_vol_call += abs(self.black_scholes(s, k_atm,  t, r, vol_atm_call, 0) - data[data['ticker']==f'RTM2P{k_atm}']['last'].values[0]) <= 0.03
                check_vol_call += abs(self.black_scholes(s, (k_atm + 1), t, r, vol_atm_call, 1) - data[data['ticker']==f'RTM2C{k_atm+1}']['last'].values[0]) <= 0.03
                check_vol_call += abs(self.black_scholes(s, (k_atm + 1), t, r, vol_atm_call, 0) - data[data['ticker']==f'RTM2P{k_atm+1}']['last'].values[0]) <= 0.03
                check_vol_call += abs(self.black_scholes(s, (k_atm + 2), t, r, vol_atm_call, 1) - data[data['ticker']==f'RTM2C{k_atm+2}']['last'].values[0]) <= 0.03
                check_vol_call += abs(self.black_scholes(s, (k_atm + 2), t, r, vol_atm_call, 0) - data[data['ticker']==f'RTM2P{k_atm+2}']['last'].values[0]) <= 0.03

                check_vol_put += abs(self.black_scholes(s, k_atm, t, r, vol_atm_put, 1) - data[data['ticker']==f'RTM2C{k_atm}']['last'].values[0]) <= 0.03
                check_vol_put += 1
                check_vol_put += abs(self.black_scholes(s, (k_atm + 1), t, r, vol_atm_put, 1) - data[data['ticker']==f'RTM2C{k_atm+1}']['last'].values[0]) <= 0.03
                check_vol_put += abs(self.black_scholes(s, (k_atm + 1), t, r, vol_atm_put, 1) - data[data['ticker']==f'RTM2P{k_atm+1}']['last'].values[0]) <= 0.03
                check_vol_put += abs(self.black_scholes(s, (k_atm + 2), t, r, vol_atm_put, 1) - data[data['ticker']==f'RTM2C{k_atm+2}']['last'].values[0]) <= 0.03
                check_vol_put += abs(self.black_scholes(s, (k_atm + 2), t, r, vol_atm_put, 1) - data[data['ticker']==f'RTM2P{k_atm+2}']['last'].values[0]) <= 0.03
            
            elif k_atm == 54:
                
                check_vol_call += 1
                check_vol_call += abs(self.black_scholes(s, k_atm,  t, r, vol_atm_call, 0) - data[data['ticker']==f'RTM2P{k_atm}']['last'].values[0]) <= 0.03
                check_vol_call += abs(self.black_scholes(s, (k_atm - 1), t, r, vol_atm_call, 1) - data[data['ticker']==f'RTM2C{k_atm-1}']['last'].values[0]) <= 0.03
                check_vol_call += abs(self.black_scholes(s, (k_atm - 1), t, r, vol_atm_call, 0) - data[data['ticker']==f'RTM2P{k_atm-1}']['last'].values[0]) <= 0.03
                check_vol_call += abs(self.black_scholes(s, (k_atm - 2), t, r, vol_atm_call, 1) - data[data['ticker']==f'RTM2C{k_atm-2}']['last'].values[0]) <= 0.03
                check_vol_call += abs(self.black_scholes(s, (k_atm - 2), t, r, vol_atm_call, 0) - data[data['ticker']==f'RTM2P{k_atm-2}']['last'].values[0]) <= 0.03

                check_vol_put += abs(self.black_scholes(s, k_atm, t, r, vol_atm_put, 1) - data[data['ticker']==f'RTM2C{k_atm}']['last'].values[0]) <= 0.03
                check_vol_put += 1
                check_vol_put += abs(self.black_scholes(s, (k_atm - 1), t, r, vol_atm_put, 1) - data[data['ticker']==f'RTM2C{k_atm-1}']['last'].values[0]) <= 0.03
                check_vol_put += abs(self.black_scholes(s, (k_atm - 1), t, r, vol_atm_put, 1) - data[data['ticker']==f'RTM2P{k_atm-1}']['last'].values[0]) <= 0.03
                check_vol_put += abs(self.black_scholes(s, (k_atm - 2), t, r, vol_atm_put, 1) - data[data['ticker']==f'RTM2C{k_atm-2}']['last'].values[0]) <= 0.03
                check_vol_put += abs(self.black_scholes(s, (k_atm - 2), t, r, vol_atm_put, 1) - data[data['ticker']==f'RTM2P{k_atm-2}']['last'].values[0]) <= 0.03
            
            if check_vol_call >= check_vol_put:
                vol_atm = vol_atm_call
            else:
                vol_atm = vol_atm_put
        
        
        data = self.price_option_vector(data, s, vol_atm)
        data['IV'] = vol_atm
        for i in data[(data['bs_price'] - data['last']).abs() > 0.03].index:
            data.loc[i, 'IV'] = self.implied_volatility_calculate(data.loc[i, 'ticker'], 
                                                                  s, vol_atm, data.loc[i, 'last'])
        data.drop('delta', axis=1, inplace=True)
        d1 = (np.log(s/data['strike']) + (data['IV']**2/2) * data['time'])/(data['IV'] * np.sqrt(data['time']))
        data['delta'] = -data['call']*(norm.cdf(d1)-1) + data['call'] * norm.cdf(d1)
        data['vega'] = s * np.sqrt(data['time']) * norm.pdf(d1)
        return data, vol_atm


    def price_option(self, ticker, spot, vol):
        # helper function for convenience, returns bs_price, delta
        period = int(ticker[3])
        strike = int(ticker[-2:])

        if ticker[4] == 'C':
            call = True
        else:
            call = False

        time = (period * 300 - ((self.period - 1) * 300 + self.tick))/3600

        return self.black_scholes(spot, strike, time, 0, vol, call), \
               self.delta(spot, strike, time, 0, vol, call)
    
    def implied_volatility_calculate(self, ticker, spot, old_vol, market_price):
        #helper function for convenience, returns implied volatility
        period = int(ticker[3])
        strike = int(ticker[-2:])

        if ticker[4] == 'C':
            call = True
        else:
            call = False

        time = (period * 300 - ((self.period - 1) * 300 + self.tick))/3600
        return self.implied_volatility(spot, strike, time, 0, market_price, old_vol, call)
    
    def price_option_vector(self, data, spot, vol):
        
        """
        Now requires to pass volatility
        """
            
        data['period'] = data['ticker'].apply(lambda x:int(x[3]))
        data['strike'] = data['ticker'].apply(lambda x:int(x[-2:]))
        data['call'] = data['ticker'].apply(lambda x:True if x[4] == 'C' else False)
        data['time'] = (data['period']*300 - ((self.period - 1) * 300 + self.tick))/3600
        
        d1 = (np.log(spot/data['strike']) + (vol**2/2) * data['time'])/(vol * np.sqrt(data['time']))
        d2 = (np.log(spot/data['strike']) + (-vol**2/2) * data['time'])/(vol * np.sqrt(data['time']))
        
        data['bs_price'] = -data['call'] * (-spot * norm.cdf(-d1) + data['strike'] * norm.cdf(-d2)) +\
                      data['call'] * (spot * norm.cdf(d1) - data['strike'] * norm.cdf(d2))
        
        data['delta'] = -data['call']*(norm.cdf(d1)-1) + data['call'] * norm.cdf(d1)
        
        return data
        
        ######################## Helper functions ########################
    
    def get_tick(self):
        resp = requests.get(f'{self.BASE}case', params=self.API_KEY).json()
        self.period = resp['period']
        self.tick = resp['tick']
        return resp['period'], resp['tick'] 
    
    def reverse_ticker(self, ticker):
        if ticker[4] == 'C':
            return ticker[0:4] + 'P' + ticker[5:]
        return ticker[0:4] + 'C' + ticker[5:]
    
    def other_month(self, ticker):
        if ticker[3] == '1':
            return ticker[0:3] + '2' + ticker[4:]
        else:
            return ticker[0:3] + '1' + ticker[4:]
        
     ###################### Market interaction functions ################
    
    def get_data(self):
        resp = requests.get(f'{self.BASE}securities', params=self.API_KEY)
        data = pd.DataFrame(resp.json())[['ticker', 'position', 'last', 'stop_period']]
        return data['last'][0], data['position'][0], data[1:]
    
    def post_order(self, ticker, action, quantity, type_='MARKET', price=0):
        resp = requests.post(f'{self.BASE}orders', params = {'key':self.KEY,
                                               'type':type_,
                                               'action':action,
                                               'ticker':ticker,
                                               'quantity':quantity,
                                                'price':price
                                               })
        return resp.json()
    
    def close_all_positions(self):
        
        spot, spot_pos, data = self.get_data()
        pos = data[data['position'] != 0]['position'].values
        tickers = data[data['position'] != 0]['ticker'].values
        while (pos == 0).sum() != len(pos):
            for i in range(len(pos)):
                if pos[i] > 0:
                    if pos[i] > 100:
                        trader.post_order(tickers[i], 'SELL', 100)
                        pos[i] -= 100
                    else:
                        trader.post_order(tickers[i], 'SELL', int(pos[i]))
                        pos[i] = 0
                elif pos[i] < 0:
                    if pos[i] < -100:
                        trader.post_order(tickers[i], 'BUY', 100)
                        pos[i] += 100

                    else:
                        trader.post_order(tickers[i], 'BUY', int(-pos[i]))
                        pos[i] = 0
                        
        self.close_rtm()
                                
    def close_rtm(self):
        
        spot,spot_pos, data = trader.get_data()
        
        if spot_pos > 0:
            while spot_pos > 10000:
                self.post_order('RTM', 'SELL', 10000)
                spot_pos -= 10000
            self.post_order('RTM', 'SELL', int(spot_pos))
            
        elif spot_pos < 0:
            while spot_pos < -10000:
                self.post_order('RTM', 'BUY', 10000)
                spot_pos += 10000

            self.post_order('RTM', 'BUY', int(abs(spot_pos)))
            
    def buy_rtm(self, q):
        
        if q > 0:
            while q > 10000:
                self.post_order('RTM', 'SELL', 10000)
                q -= 10000
            self.post_order('RTM', 'SELL', int(q))
            
        elif q < 0:
            while q < -10000:
                self.post_order('RTM', 'BUY', 10000)
                q += 10000
            self.post_order('RTM', 'BUY', int(abs(q)))

In [ ]:
# main function
trader = Trader_2(BASE, API_KEY, KEY, strike_prices)
trader.get_tick()
trader.get_delta_limit()
trader.get_real_volatility()

while True:

    if (trader.tick % 75 >= 70 or (trader.tick >= 290 and trader.period == 1)) and not trader.started_vol and trader.period * trader.tick < 500:
        trader.get_tick()    
        trader.open_vol()

    if trader.tick % 75 < 60 and trader.started_vol:
        trader.close_vol()


    if trader.tick % 75 <= 47 and not trader.started_arb_p2 and not trader.started_vol:
        trader.open_arb_p2()

    if trader.started_arb_p2:
        trader.close_arb_p2()


    trader.delta_hedge(full=True)

    time.sleep(0.5)

    if trader.tick >= 280 and trader.period == 2:
        trader.end_straddle() #sleep so doesn't start hedging
        time.sleep(20)


    trader.get_tick()
    trader.get_real_volatility()